In [ ]:
#!/usr/bin/env python3
#coding=utf-8

#  ______               _         _____      _ _ _                  _                 _         _    _  _____ _____
# |  ____|             | |       |  __ \    | (_| |                (_)               | |       | |  | |/ ____|  __ \
# | |__   ___  ___ ___ | | __ _  | |__) ___ | |_| |_ ___  ___ _ __  _  ___ __ _    __| | __ _  | |  | | (___ | |__) |
# |  __| / __|/ __/ _ \| |/ _` | |  ___/ _ \| | | __/ _ \/ __| '_ \| |/ __/ _` |  / _` |/ _` | | |  | |\___ \|  ___/
# | |____\__ | (_| (_) | | (_| | | |  | (_) | | | ||  __| (__| | | | | (_| (_| | | (_| | (_| | | |__| |____) | |
# |______|___/\___\___/|_|\__,_| |_|   \___/|_|_|\__\___|\___|_| |_|_|\___\__,_|  \__,_|\__,_|  \____/|_____/|_|
# ___________________________________________________________________________________________________________________
#                                           Testes com Outputs de fora da rede
# ___________________________________________________________________________________________________________________
#
#  9848836 - Bruna Okura
#  11913194 - Lucas Lima Emerêncio
#  11805742 - Yann Gazzolla dos Santos
# ___________________________________________________________________________________________________________________

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

Mounted at /content/gdrive


In [2]:
import os
import numpy as np
import torch
from PIL import Image
from torch import nn
from torchvision import models, transforms
import subprocess
from matplotlib import pyplot as plt
import cv2
import json

# INICIALIZANDO REDE CLASSIFICADORA

In [3]:
# Configurações iniciais
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Parâmetros de entrada
model_name = "vgg16"
weights_path = "/content/gdrive/MyDrive/TCC/Rede_Definitiva/New_detection/Weights6/vgg16_aug_pt.pth"
test_dir = "/content/gdrive/MyDrive/TCC/Rede_Definitiva/New_detection/Test"
quiet_mode = True

# Carregar o modelo
if model_name == "vgg16":
    model = models.vgg16(weights=None)
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, 6)
else:
    raise ValueError(f"Modelo '{model_name}' não suportado.")

model = torch.nn.DataParallel(model)
model.to(device)

# Carregar pesos com `weights_only=True` para evitar avisos de segurança
model.load_state_dict(torch.load(weights_path, map_location=device, weights_only=True))
model.eval()

# Transformações de pré-processamento
data_transforms = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# Identificar classes a partir dos nomes das pastas no diretório de teste
class_names = ["Aedes_albopictus", "Aedes_fulvus", "Aedes_scapularis", "Aedes_serratus", "Anopheles_bellator", "Mansonia_humeralis"]
classes = {indice : name  for indice, name in enumerate(class_names)}


# INICIALIZANDO REDE DETECTORA

In [ ]:
# clone darknet repo
!git clone https://github.com/LucasEmerencio/darknet_mosquito

%cd darknet_mosquito

!make

## HELPER FUNCTIONS

In [5]:
def get_coordinates_json(result_file):
    coordinates = []

    with open(result_file, 'r') as file:
        data = json.load(file)  # Load the JSON data

        for frame in data:
            frame_id = frame["frame_id"]
            for obj in frame["objects"]:
                relative_coords = obj["relative_coordinates"]

                # Extract the relative coordinates
                center_x = relative_coords["center_x"]
                center_y = relative_coords["center_y"]
                width = relative_coords["width"]
                height = relative_coords["height"]

                # Add the coordinates and frame ID to the list
                coordinates.append({
                    "frame_id": frame_id,
                    "center_x": center_x,
                    "center_y": center_y,
                    "width": width,
                    "height": height
                })

    return coordinates

In [6]:
def crop_and_resize_image(path, cx, cy, range_x, range_y, crop_path = None):
    # Load image
    image = cv2.imread(path)
    if image is None:
        print("Error: Image not found at the specified path.")
        return

    img_height, img_width = image.shape[:2]

    #Coordenadas Absolutas
    cx = int(cx * img_width)
    cy = int(cy * img_height)

    range_x = int(range_x * img_width)
    range_y = int(range_y * img_height)

    # Calculo do vértice superior esquerdo da caixa de crop
    x_start = int(cx - range_x / 2)
    y_start = int(cy - range_y / 2)

    # Assegurando que as dimensões de crop respeitam os limites da imagem
    x_start = max(0, x_start)
    y_start = max(0, y_start)
    x_end = min(img_width, x_start + range_x)
    y_end = min(img_height, y_start + range_y)

    # Crop the image
    cropped_image = image[y_start:y_end, x_start:x_end]

    # Resize cropped image to 512x512
    resized_image = cv2.resize(cropped_image, (224, 224), interpolation=cv2.INTER_CUBIC)

    #Salvamos a imagem com crop no diretório especificado se o caminho for passado para função
    if crop_path is not None:
      jpeg_filename = os.path.join(crop_path, f"{os.path.splitext(os.path.basename(path))[0]}.jpg")
      cv2.imwrite(jpeg_filename, resized_image)

    return resized_image

In [7]:
def detect_and_crop(path):

  try:
    result = subprocess.run(["./darknet", "detector", "test", "data/obj.data", "cfg/yolov3_custom2.cfg", "/content/gdrive/MyDrive/TCC/REPRODUÇÃO_DE_RESULTADOS/backup/yolov3_custom2_last.weights", path, "-thresh", "0.2", "-ext_output", "-out", "result.txt"], shell = False, check = True)
  except subprocess.CalledProcessError as e:
    print(f"Error during batch processing: {e}")

  coordinates = get_coordinates_json("result.txt")
  coordinates = coordinates[0]

  if coordinates["frame_id"] is None:
    return None
  else:
    cx, cy, width, height = (
      coordinates["center_x"],
      coordinates["center_y"],
      coordinates["width"],
      coordinates["height"]
    )
    #crop_path é facultativo, se for omitido as images não serão salvas em outro diretório
    imagem = crop_and_resize_image(path, cx, cy, width, height)

    return imagem

In [8]:
# define helper functions
def imShow(path):
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(5, 5)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# REDE CONCATENADA

In [ ]:
while (True):
  try:
    teste = input("Digite um caminho para uma imagem válida: ")
    if os.path.exists(teste):
      print(f"Caminho do arquivo encontrado")

      valid_extensions = ['.jpg', '.jpeg']
      if (any(teste.lower().endswith(ext) for ext in valid_extensions)):
        print(f"Arquivo é uma imagem")
        imagem = detect_and_crop(teste)

        if imagem is None:
          print(f"Algum erro no processo, confira a imagem antes de colocar o caminho")
        else:
          cv2.imwrite("resultado.jpg", imagem)
          imagem = Image.open("resultado.jpg")
          input_tensor = data_transforms(imagem).unsqueeze(0).to(device)

          confidence = 6.0
          with torch.no_grad():
              output = model(input_tensor)
              print(f"logits antes do threshold = {output}")
              output = torch.where(output > confidence, output, torch.tensor(0.0))

              if (torch.count_nonzero(output).item() != 0):
                predicted_label = torch.argmax(output).item()
                #print(f"logits após threshold = {output}")
                print(f"Resultado da Classificação : {classes[predicted_label]} \n")
              else:
                print(f"Baixa confiança na predição, tente utilizar outra foto. Na eventualidade de uma segunda negativa, considerar que a espécie não está no banco de treinamento: \n DESCONHECIDA \n")
                    #imShow("resultado.jpg")

      else:
        raise ValueError("Caminho não é de uma imagem \n")

    else:
      raise OSError("Caminho inválido \n")
  except OSError as erro:
    print(erro)
  except ValueError as erro:
    print(erro)
